## Tutorial: 

#### Configure the Environment

Import the chi example API calls, set the project name and region, and set various names and attributes to use in the tutorial. 

In [ ]:
import json
import os
import chi

from chi.server import *
from chi.lease import *
from chi.network import *

from datetime import datetime, timedelta
from dateutil import tz

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'
VERBOSE=False  #Set to true for extra output

#Config with your project and sites
chi.set('project_name', 'CH-XXXXXX')
chi.use_site('CHI@UC')

#Insert keypair name
key_name = 'my_chameleon_key'  # Change to your keypair

#GENI Pem file
geni_pem_file='~/my_geni_file.pem'  # Change to your GENI file location

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")+'12'
server_name = username+'Server'
network_name = username+'Net'
subnet_name = username+'subnet'
router_name = username+'Router'
lease_name = username+'Lease'

#Server Config
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_haswell"
server_count=1

#Network Config
physical_network='exogeni'
cidr='192.168.42.0/24'

uc_allocation_start='192.168.42.101'
uc_allocation_end='192.168.42.150'
uc_gateway='192.168.42.100'
tacc_allocation_start='192.168.42.201'
tacc_allocation_end='192.168.42.250'
tacc_gateway='192.168.42.200'

## Create Chicago Network and Server

#### Create a Lease at Chicago

In [ ]:
#Set the region
chi.use_site('CHI@UC')    # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

In [ ]:
chi.use_site('CHI@UC')

#Get the lease by name
uc_lease = get_lease(get_lease_id(lease_name))
    
#Print the lease info
if VERBOSE:
    print(json.dumps(uc_lease, indent=2))
else:
    print('uc_lease: ' + uc_lease['name'] + ', ' + uc_lease['id'])

#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [ ]:
#Get the lease by name
uc_lease = get_lease(get_lease_id(lease_name))

uc_compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', uc_lease['reservations']))[0]['id']
uc_network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', uc_lease['reservations']))[0]['id']
uc_floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', uc_lease['reservations']))[0]['id']

print("uc_compute_reservation_id: " + uc_compute_reservation_id)
print("uc_network_reservation_id: " + uc_network_reservation_id)
print("uc_floatingip_reservation_id: " + uc_floatingip_reservation_id)

#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [ ]:
#Get the network
uc_network = get_network(get_network_id(network_name))
uc_network_id = uc_network['id']
uc_network_vlan = uc_network['provider:segmentation_id']

if VERBOSE:
    print(json.dumps(uc_network, indent=2))
else:
    print('uc_network: ' + uc_network['name'] + ', ' + uc_network['id'])
    print('uc segmenation ID/VLAN: ' + str(uc_network_vlan))

#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [ ]:
subnet = create_subnet(subnet_name, 
                       uc_network_id, 
                       cidr=cidr,
                      allocation_pool_start=uc_allocation_start,
                      allocation_pool_end=uc_allocation_end,
                      gateway_ip=uc_gateway)



#print(json.dumps(subnet, indent=2))

In [ ]:
if VERBOSE:
    print(json.dumps(subnet, indent=2))
else:
    print('Subnet: ' + subnet['name'] + ', ' + subnet['id'])

#### Add a Router

TODO: add links here

In [ ]:
router = create_router(router_name, gw_network_name='public')
#print(json.dumps(router, indent=2))

if VERBOSE:
    print(json.dumps(router, indent=2))
else:
    print('Router: ' + router['name'] + ', ' + router['id'])

#### Attach the Router and Subnet

TODO: Add links here

In [ ]:
add_subnet_to_router(get_router_id(router_name), get_subnet_id(subnet_name))

## Create TACC Network and Server

#### Create a Lease at Chicago

In [ ]:
node_type="compute_cascadelake"

#Set the region
chi.use_site('CHI@TACC')    # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

In [ ]:
chi.use_site('CHI@TACC')

#Get the lease by name
tacc_lease = get_lease(get_lease_id(lease_name))
    
#Print the lease info
if VERBOSE:
    print(json.dumps(tacc_lease, indent=2))
else:
    print('tacc_lease: ' + tacc_lease['name'] + ', ' + tacc_lease['id'])

#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [ ]:
#Get the lease by name
tacc_lease = get_lease(get_lease_id(lease_name))

tacc_compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', tacc_lease['reservations']))[0]['id']
tacc_network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', tacc_lease['reservations']))[0]['id']
tacc_floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', tacc_lease['reservations']))[0]['id']

print("tacc_compute_reservation_id: " + tacc_compute_reservation_id)
print("tacc_network_reservation_id: " + tacc_network_reservation_id)
print("tacc_floatingip_reservation_id: " + tacc_floatingip_reservation_id)

#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [ ]:
chi.use_site('CHI@TACC')    # Optional, defaults to 'CHI@UC'

#Get the network
tacc_network = get_network(get_network_id(network_name))
tacc_network_id = tacc_network['id']
tacc_network_vlan = tacc_network['provider:segmentation_id']

if VERBOSE:
    print(json.dumps(tacc_network, indent=2))
else:
    print('tacc_network: ' + tacc_network['name'] + ', ' + tacc_network['id'])
    print('tacc segmenation ID/VLAN: ' + str(tacc_network_vlan))

#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [ ]:
subnet = create_subnet(subnet_name, 
                       tacc_network_id, 
                       cidr=cidr, 
                      allocation_pool_start=tacc_allocation_start,
                      allocation_pool_end=tacc_allocation_end,
                      gateway_ip=tacc_gateway)

 

#print(json.dumps(subnet, indent=2))

In [ ]:
if VERBOSE:
    print(json.dumps(subnet, indent=2))
else:
    print('Subnet: ' + subnet['name'] + ', ' + subnet['id'])

#### Add a Router

TODO: add links here

In [ ]:
router = create_router(router_name, gw_network_name='public')
#print(json.dumps(router, indent=2))

if VERBOSE:
    print(json.dumps(router, indent=2))
else:
    print('Router: ' + router['name'] + ', ' + router['id'])

#### Attach the Router and Subnet

TODO: Add links here

In [ ]:
add_subnet_to_router(get_router_id(router_name), get_subnet_id(subnet_name))

## Start the Servers

#### Start the UC Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [ ]:
chi.use_site('CHI@UC')  
#create the server
server = create_server(server_name, 
                       reservation_id=uc_compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [ ]:
chi.use_site('CHI@UC')
server_id = get_server_id(server_name)
uc_floating_ip = associate_floating_ip(server_id)

uc_fixed_ip = get_server(server_id).interface_list()[0].to_dict()["fixed_ips"][0]["ip_address"]

print('Floating IP: ' + str(uc_floating_ip))
print('Fixed IP: ' + str(uc_fixed_ip))

#### Start the TACC Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [ ]:
chi.use_site('CHI@TACC')

print("server_name " + server_name)
print("tacc_compute_reservation_id " + tacc_compute_reservation_id)
print("key_name " + key_name) 
print("network_name " + network_name)
print("image_name " + image_name)
print("flavor_name " + flavor_name)

#create the server
server = create_server(server_name, 
                       reservation_id=tacc_compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [ ]:
chi.use_site('CHI@TACC')  

server_id = get_server_id(server_name)
tacc_floating_ip = associate_floating_ip(server_id)
tacc_fixed_ip = get_server(server_id).interface_list()[0].to_dict()["fixed_ips"][0]["ip_address"]

print('Floating IP: ' + str(tacc_floating_ip))
print('Fixed IP: ' + str(tacc_fixed_ip))

## Stitch the Circuit using ExoGENI

Note: The ExoGENI
steps require a valid GENI certificate at the path specified and a public/private keypair in ~/.ssh (run ssh-keygen with default inputs)

#### Create the Circuit

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch create -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

#### Check the Status of the Circuit

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch status -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

## Access the Circuit

#### Create scripts

In [ ]:
import paramiko

# This script installs vim and iperf3 onto our UC node and runs an iperf3 server as a Daemon.
# vim is a text editor used by bash.
# iperf3 is a host-tuning tool that allows us to observe data transfer between servers. The iperf server running out of UC will be accessed by our iperf command on TACC node.
# As a Daemon, this server will run in the background until we manually disable it. This is necessary for us to be able to access the server on our TACC node.
uc_script = '#!/bin/bash'   '\n' \
    'yum install vim iperf3 -y'   '\n' \
    'iperf3 -s -D'   '\n'

# This script manually kills our iperf3 Daemon server which we will have launched previously.
uc_script_cleanup = '#!/bin/bash'   '\n' \
    'pkill iperf'   '\n'

# This script installs vim and iperf3 onto our TACC node, like above, and connects to the iperf3 server we will have set up on UC to get information on data transfers.
tacc_script = '#!/bin/bash'   '\n' \
    'yum install vim iperf3 -y'   '\n' \
    'iperf3 -t 30 -i 5 -c '+uc_fixed_ip+   '\n'

#### Wait TCP Accessible

In [ ]:
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())

# The time it takes for your server to be TCP-ready can vary greatly.
# This function, chi.server.wait_for_tcp, halts your cell's progress until the IP is tcp-accessible on the SSH port.
# The timeout is set to 45 minutes (60*45 seconds) due to greatly varying start-up times.
wait_for_tcp(uc_floating_ip, '22', timeout=(60*45))
print('UC accessible')
wait_for_tcp(tacc_floating_ip, '22', timeout=(60*45))
print('TACC accessible')

uc_key_location = "~/my_chameleon_key_uc.pem"  # Change to your UC key location
tacc_key_location = "~/my_chameleon_key_uc.pem"  # Change to your TACC key location

uc_key = paramiko.RSAKey.from_private_key_file(uc_key_location)
tacc_key = paramiko.RSAKey.from_private_key_file(tacc_key_location)

#### Run UC Script

In [ ]:
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
client.connect(uc_floating_ip,username='cc',pkey = uc_key) # Sometimes this line may throw an error. Wait a few minutes and then try again!

stdin_uc, stdout_uc, stderr_uc = client.exec_command('echo \"' + uc_script + '\" > script.sh; chmod +x script.sh; sudo ./script.sh')

#### Run TACC Script

In [ ]:
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
client.connect(tacc_floating_ip,username='cc',pkey = tacc_key) # Sometimes this line may throw an error. Wait a few minutes and then try again!

stdin_tacc, stdout_tacc, stderr_tacc = client.exec_command('echo \"' + tacc_script + '\" > script.sh; chmod +x script.sh; sudo ./script.sh')
print (stdout_tacc.read().decode())
print (stderr_tacc.read().decode())

#### Close Paramiko Client

In [ ]:
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
client.connect(uc_floating_ip,username='cc',pkey = uc_key) # Sometimes this line may throw an error. Wait a few minutes and then try again!

stdin, stdout, stderr = client.exec_command('echo \"' + uc_script_cleanup + '\" > script.sh; chmod +x script.sh; sudo ./script.sh')

client.close()

## Tuning

On your first run of iperf3, the results are not impressive. The nodes that we have set up have network cards supporting bandwidth up to 10 Gbits/sec. By modifying some network settings in our nodes, we can approach bandwidth capacity.

TCP buffers are buffers on each side of a transfer connection that hold data before it is transfered - and when this buffer fills up, the recieving node will alert the sending node that it cannot send any more data until this cache is cleared. The Linux TCP buffers automatically set by our nodes' OSes are too small to utilize our bandwidth capabilities. We can modify these manually using the cells below. 

These cells establish a `host_tuning` variable with which we want to update our nodes. Updating our nodes with these settings will increase the size of the TCP buffer on each side (sending and recieving) of the transfer, alongside other subtle changes. 

In [ ]:
host_tuning = '# allow testing with buffers up to 64MB'   '\n' \
    'net.core.rmem_max = 67108864'   '\n' \
    'net.core.wmem_max = 67108864'   '\n' \
    '# increase Linux autotuning TCP buffer limit to 32MB'   '\n' \
    'net.ipv4.tcp_rmem = 4096 87380 33554432'   '\n' \
    'net.ipv4.tcp_wmem = 4096 65536 33554432'   '\n' \
    '# recommended default congestion control is htcp'   '\n' \
    'net.ipv4.tcp_congestion_control=htcp'   '\n' \
    '# recommended for hosts with jumbo frames enabled'   '\n' \
    'net.ipv4.tcp_mtu_probing=1'   '\n' \
    '# recommended to enable "fair queueing"'   '\n' \
    'net.core.default_qdisc = fq'

In [ ]:
host_tuning = '# allow testing with buffers up to 128MB'   '\n' \
    'net.core.rmem_max = 134217728'   '\n' \
    'net.core.wmem_max = 134217728'   '\n' \
    '# increase Linux autotuning TCP buffer limit to 64MB'   '\n' \
    'net.ipv4.tcp_rmem = 4096 87380 67108864'   '\n' \
    'net.ipv4.tcp_wmem = 4096 65536 67108864'   '\n' \
    '# recommended default congestion control is htcp'   '\n' \
    'net.ipv4.tcp_congestion_control=htcp'   '\n' \
    '# recommended for hosts with jumbo frames enabled'   '\n' \
    'net.ipv4.tcp_mtu_probing=1'   '\n' \
    '# recommended to enable fair queueing'   '\n' \
    'net.core.default_qdisc = fq'

Running this cell will update both of your nodes with your `host_tuning` variable's settings. Keep in mind that both variables have the same name, so whichever one of the above cells you ran last is the one whose settings will be used on running the below cell.

After running the below cell, feel free to go back above and re-run your iperf3 cells in order to observer the changes.

In [ ]:
host_tuning_script = '#!/bin/bash'   '\n' \
    'sudo sh -c \'echo -e "' + host_tuning + '" > /etc/sysctl.conf\''   '\n' \
    'sudo sysctl --system'

client = paramiko.SSHClient()
client.load_system_host_keys()

client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
client.connect(uc_floating_ip,username='cc',pkey = uc_key) # Sometimes this line may throw an error. Wait a few minutes and then try again!

stdin_uc, stdout_uc, stderr_uc = client.exec_command(host_tuning_script)

client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
client.connect(tacc_floating_ip,username='cc',pkey = tacc_key) # Sometimes this line may throw an error. Wait a few minutes and then try again!

stdin_tacc, stdout_tacc, stderr_tacc = client.exec_command(host_tuning_script)
print (stdout_tacc.read().decode())
print (stderr_tacc.read().decode())

client.close()

### Modify scripts

Now that our TCP buffer size has been increased, we can up the TCP window size in our data transfer. TCP window size is a measurement of how much data the recieving end of our data transfer will accept at any given size, and is restricted by the TCP buffer size. By default, the TCP window size is set to approximately 64 kilobytes. By rewriting our TACC script with the `-w` tag, we can alter the window size directly. This script uses 64 megabits -- 8000 kilobytes, or 125x the size.

After running updating this script, go ahead and run your iperf3 test again. Your bandwidth should now be much closer to 10 Gbits/sec.

In [ ]:
# Something something window size yeah
tacc_script = '#!/bin/bash'   '\n' \
    'yum install vim iperf3 -y'   '\n' \
    'iperf3 -t 30 -i 5 -c '+uc_fixed_ip+' -w 64m'   '\n'

## Clean Up Resources

### Delete Stitched Circuit using ExoGENI

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch delete -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

### Delete TACC Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [ ]:
chi.use_site('CHI@TACC')

In [ ]:
delete_server(get_server_id(server_name))

#### De-configure Network
TODO: break up into steps

In [ ]:
remove_subnet_from_router(router['id'], subnet['id'])

In [ ]:
delete_router(router['id'])

In [ ]:
delete_subnet(subnet['id'])

In [ ]:
delete_network(tacc_network['id'])

#### Release Lease

In [ ]:
delete_lease(tacc_lease['id'])

### Delete UC Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [ ]:
chi.use_site('CHI@UC')

In [ ]:
delete_server(get_server_id(server_name))

#### De-configure Network
TODO: break up into steps

In [ ]:
remove_subnet_from_router(get_router_id(router_name), get_subnet_id(subnet_name))

In [ ]:
delete_router(get_router_id(router_name))

In [ ]:
delete_subnet(get_subnet_id(subnet_name))

In [ ]:
delete_network(uc_network['id'])

#### Release Lease

In [ ]:
delete_lease(uc_lease['id'])